# Progetto di Social Computing

a.a. 2022-2023

## Attività preliminari

### Librerie e costanti

In [20]:
# Caricamento delle librerie
import os, json, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Cartelle di salvataggio
data_folder = "./data"
out_folder = "./out"

### Funzioni

In [2]:
# Salvataggio in locale
def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent = 4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

In [4]:
# Lettura da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

## Caricamento del dataset

Si carica il dataset fornito, frammento della combinazione dei dataset [FEVER](https://fever.ai/dataset/fever.html) e [e-FEVER](https://doi.org/10.3929/ethz-b-000453826).

In [32]:
# Si carica e si mostra il dataset fornito
df = pd.read_csv("./group_9.csv")
display(df)

,id,statement,explanation_human,explanation_model,label
0,51526,Hush (2016 film) was produced by Jason Blum.,Hush (2016 film) was produced by Trevor Macy a...,The evidence says that the film was produced b...,REFUTES
1,77465,Winter's Tale was released in 1987.,Winter's Tale was released in 2014.,The claim is that Winter's Tale was released i...,REFUTES
2,166632,Anne Rice was born in the United States of Ame...,"Anne Rice was born in New Orleans, Louisiana, ...",The claim is that Anne Rice was born in the Un...,SUPPORTS


## Creazione degli HITs

Con il dataset fornito, vogliamo creare creare dodici HITs aventi le seguenti caratteristiche:

1. contiene 3 elementi;
2. ogni elemento è dotato di 4 attributi:
   1. `id`: identificatore dello statement;
   2. `statement`: testo dello statement;
   3. `explanation`: testo della spiegazione;
   4. `label`: etichetta della spiegazione.
3. per due elementi su tre vale che `explanation` = `explanation_model`;
4. per un elemento su tre vale che `explanation` = `explanation_human`;
5. la posizione dei tre elementi deve essere casuale.

In [14]:
# Funzione per l'interpretazione delle righe del dataset
def parse_row(row, isGold = False):
    parsed = {
        "id" : row.id,
        "statement" : row.statement,
        "explanation" : row.explanation_model,
        "label" : row.label,
        "isGold" : isGold
    }

    # Se è una "domanda d'oro", la spiegazione deve essere quella fornita da un essere umano
    if isGold:
        parsed["explanation"] = row.explanation_human
    
    return parsed

In [31]:
# Creiamo tutte le possibili permutazioni dei 3 elementi secondo quanto stabilito dalla consegna
all_HITs = []

# Iteriamo sulle spiegazioni fornite dai modelli di machine learning
for model_exp in df.itertuples():
    model_HITs = []

    # Iteriamo sugli altri "statements"
    for other_stat in df.itertuples():
        if other_stat.id != model_exp.id:
            # Inseriamo la versione con la "domanda d'oro" per la spiegazione del modello (model_exp)
            other_model_HIT = [parse_row(model_exp), parse_row(model_exp, True), parse_row(other_stat)]
            # Inseriamo la versione con la "domanda d'oro" per questo statement (other_stat)
            other_gold_HIT = [parse_row(model_exp), parse_row(other_stat, True), parse_row(other_stat)]

            # Riordiniamo pseudo-casualmente gli elementi
            random.shuffle(other_model_HIT)
            random.shuffle(other_gold_HIT)
            # Aggiungiamo le HIT create a quelle relative a questa spiegazione (model_exp)
            model_HITs.append(other_model_HIT)
            model_HITs.append(other_gold_HIT)
    
    # Si concatena il tutto a tutti gli HITs possibili
    all_HITs += model_HITs

# Riposizioniamo gli elementi della lista in maniera pseudo-causale
random.shuffle(all_HITs)

# Salviamo la lista degli HIT creata
serialize_json(data_folder, "all_HITs.json", all_HITs)

Data serialized to path: ./data/all_HITs.json
